# **2110443 - Computer Vision (2021/2)**
#**Lab 8 - Convolutional Neural Network [Homework]** <br>
In this lab, we will learn how to use Convolutional Neural Network to perform image classification in the provided real world dataset using PyTorch. This notebook includes both coding and written questions. Please hand in this notebook file with all outputs and your answer.

**Collaboration is encouraged in this course.** You must turn in your own write ups of all problems. If you collaborate with others, you must put the names and ids of the students you worked with in below block.

Collaboration List:
- ...
- ...


# Assignment 1 : Food Image Classification
![alt text](https://cdn.pixabay.com/photo/2015/08/26/10/58/the-pork-fried-rice-made-908333_1280.jpg)

Classify 50 food menus from Chula-Food-50 dataset
 

In this assignment you have to replace YOUR_STUDENT_ID_WITH21 variable with your student id (in integer). There will be 2 sets of data: train and test 

By using the knowledge from the lab and lecture, you have to design your own CNN food image classification model and tested on unknown label dataset!



Scoreboard URL : https://www.piclab.ai/classes/cv2021/lab8/scoreboard

In [ ]:
import random
import glob
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import matplotlib
import matplotlib.font_manager as fm
from PIL import Image

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models as models
import albumentations, albumentations.pytorch
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

##### Add Thai font ######

!wget https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
fm.fontManager.ttflist += fm.createFontList(['thsarabunnew-webfont.ttf'])
matplotlib.rc('font', family='TH Sarabun New')

##### Don't forget to put your ID here (in integer) ####
YOUR_STUDENT_ID_WITH21 = 6130112321 
#######################################################

def isStudentIDValid(studentID):
  strID = str(studentID)
  isEndWith21 = strID.endswith('21')
  isLengthOK = len(strID) == 10
  if isEndWith21 and isLengthOK:
    return True
  else:
    return False

print('Student ID status:',isStudentIDValid(YOUR_STUDENT_ID_WITH21))

--2022-03-19 05:15:19--  https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Phonbopit/sarabun-webfont/master/fonts/thsarabunnew-webfont.ttf [following]
--2022-03-19 05:15:19--  https://raw.githubusercontent.com/Phonbopit/sarabun-webfont/master/fonts/thsarabunnew-webfont.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98308 (96K) [application/octet-stream]
Saving to: ‘thsarabunnew-webfont.ttf’

thsarabunnew-webfon 100%[===================>]  96.00K  --.-KB/s    in 0.004s  

2022-03-19 05:15:19 (21.2 MB/s) - ‘t

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.


## Your model description goes here: ###


1.   Use EfficientNetV2-s as backbone of the model
2.   Create new linear layer as classifier layer instead of existing classifier layer
3.   Use transfer learning technique to train with food dataset
4.   Use several image augmentation techniques

##GPU Status Check

In [ ]:
!nvidia-smi

Sat Mar 19 05:15:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download and inspect Chula-Food-50 dataset

In [ ]:
!wget  -O chula-food-50.zip https://piclab.ai/classes/cv2021/Chula-food-50.zip
!unzip -qo chula-food-50.zip

--2022-03-19 05:15:20--  https://piclab.ai/classes/cv2021/Chula-food-50.zip
Resolving piclab.ai (piclab.ai)... 161.200.92.132
Connecting to piclab.ai (piclab.ai)|161.200.92.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 968269678 (923M) [application/zip]
Saving to: ‘chula-food-50.zip’

chula-food-50.zip   100%[===================>] 923.41M  12.0MB/s    in 78s     

2022-03-19 05:16:39 (11.8 MB/s) - ‘chula-food-50.zip’ saved [968269678/968269678]



In [ ]:
### Helper function to display image from dataset ###
def getImageFromDataset(dataset, idx):
  sampleImage, sampleLabel = dataset.__getitem__(idx)
  ### Revert transformation ###
  sampleImage = ((sampleImage.permute(1,2,0).numpy() * np.array([0.229, 0.224, 0.225])) + np.array([0.485, 0.456, 0.406]))*255
  sampleImage = sampleImage.astype(np.uint8)
  sampleClassName = dataset.classes[sampleLabel]
  return sampleImage, sampleClassName

In [ ]:
'''
#### FILL Any Augmenetation HERE ####
transformTrain = transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

transformVal =  transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
### Load Dataset ###
foodTrainDataset = 
'''


'\n#### FILL Any Augmenetation HERE ####\ntransformTrain = transforms.Compose([\n        transforms.Resize(size=(224,224)),\n        transforms.ToTensor(),\n        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])\n\ntransformVal =  transforms.Compose([\n        transforms.Resize(size=(224,224)),\n        transforms.ToTensor(),\n        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])\n### Load Dataset ###\nfoodTrainDataset = \n'

In [ ]:
'''
### Split train and validation ###
TEST_SIZE = 0.1
SEED = 42
BATCH_SIZE = 16
# generate indices: instead of the actual data we pass in integers instead
train_indices, test_indices, _, _ = train_test_split(
    range(len(foodTrainDataset)),
    foodTrainDataset.targets,
    stratify=foodTrainDataset.targets,
    test_size=TEST_SIZE,
    random_state=SEED
)

# generate subset based on indices
train_split = Subset(foodTrainDataset, train_indices)
test_split = Subset(foodTrainDataset, test_indices)
'''

'\n### Split train and validation ###\nTEST_SIZE = 0.1\nSEED = 42\nBATCH_SIZE = 16\n# generate indices: instead of the actual data we pass in integers instead\ntrain_indices, test_indices, _, _ = train_test_split(\n    range(len(foodTrainDataset)),\n    foodTrainDataset.targets,\n    stratify=foodTrainDataset.targets,\n    test_size=TEST_SIZE,\n    random_state=SEED\n)\n\n# generate subset based on indices\ntrain_split = Subset(foodTrainDataset, train_indices)\ntest_split = Subset(foodTrainDataset, test_indices)\n'

## Dataset Visualization

In [ ]:
'''
image1, image1ClassName = getImageFromDataset(foodTrainDataset, 0)
image2, image2ClassName = getImageFromDataset(foodTrainDataset, 1600)

_, figure = plt.subplots(1,2)

figure[0].imshow(image1,cmap='gray')
figure[0].set_title(image1ClassName)
figure[1].imshow(image2,cmap='gray')
figure[1].set_title(image2ClassName)
plt.show()
'''

"\nimage1, image1ClassName = getImageFromDataset(foodTrainDataset, 0)\nimage2, image2ClassName = getImageFromDataset(foodTrainDataset, 1600)\n\n_, figure = plt.subplots(1,2)\n\nfigure[0].imshow(image1,cmap='gray')\nfigure[0].set_title(image1ClassName)\nfigure[1].imshow(image2,cmap='gray')\nfigure[1].set_title(image2ClassName)\nplt.show()\n"

## Define CNN network for food classification
Hint
1. You can freely uses any structure/pretrained model to do this homework but don't forgot to cited them in this notebook.

   A very big collection of pretrained model can be found here : https://github.com/rwightman/pytorch-image-models

2. Don't forget to change mean and std in the pre-processing to match with your pretrained model.

In [ ]:
'''
#### Design you network here ####
class foodNet(nn.Module):
  def __init__(self,)
    super(foodNet, self).__init__()
    ### Layers goes here ###
  def forward(self, input):
    ### Conntections goes here ###
    return ?
'''

'\n#### Design you network here ####\nclass foodNet(nn.Module):\n  def __init__(self,)\n    super(foodNet, self).__init__()\n    ### Layers goes here ###\n  def forward(self, input):\n    ### Conntections goes here ###\n    return ?\n'

## Construct the model, optimizer and loss function

In [ ]:
'''
#### FILL HERE ####
foodNet.cuda()

criterion = 
optimizer = 
scheduler =

foodTrainDatasetLoader = 
foodValDatasetLoader = 
'''

'\n#### FILL HERE ####\nfoodNet.cuda()\n\ncriterion = \noptimizer = \nscheduler =\n\nfoodTrainDatasetLoader = \nfoodValDatasetLoader = \n'

## Train the model

In [ ]:
'''
### Train and test helper function ###
def testModel(testDatasetLoader, net):
  net.eval()
  correctImages = 0
  totalImages = 0
  allLabels = []
  allPredicted = []
  testingProgressbar = tqdm(enumerate(testDatasetLoader), total=len(testDatasetLoader), ncols=100)
  with torch.no_grad():
    for batchIdx, batchData in testingProgressbar:
      images, labels = batchData
      
      images, labels = images.cuda(), labels.cuda()
      outputs = net(images)
      _, predicted = torch.max(outputs, 1)

      correctImages += (predicted == labels).sum().item()
      totalImages += labels.size(0)

      accumulateAccuracy = round((correctImages/totalImages)*100,4)
      testingProgressbar.set_description("Testing accuracy: {}".format(accumulateAccuracy ) )
    
      allLabels.append(labels)
      allPredicted.append(predicted)
  allLabels = torch.cat(allLabels).cpu().numpy()
  allPredicted = torch.cat(allPredicted).cpu().numpy()
  return correctImages, totalImages, allLabels, allPredicted

def trainAndTestModel(trainDatasetLoader, testDatasetLoader, net, optimizer,scheduler, criterion, trainEpoch):
  
  bestAccuracy = 0
  correctImages = 0
  totalImages = 0
  for currentEpoch in tqdm(range(trainEpoch), desc='Overall Training Progress:', ncols=100):
    trainingLoss = 0.0
    net.train()
    print('Epoch',str(currentEpoch+1),'/',str(trainEpoch))
    trainingProgressbar = tqdm(enumerate(trainDatasetLoader), total=len(trainDatasetLoader), ncols=100)
    for batchIdx, batchData in trainingProgressbar:
      images, labels = batchData
      images, labels = images.cuda(), labels.cuda()

      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = net(images)
      loss = criterion(outputs, labels)
    
      _, predicted = torch.max(outputs, 1)
      correctImages += (predicted == labels).sum().item()
      totalImages += labels.size(0)
    
      loss.backward()
      optimizer.step()
      

      trainingLoss += loss.item()
      accumulateAccuracy = round((correctImages/totalImages)*100,4)
      trainingProgressbar.set_description("Training accuracy: {} loss: {}".format(accumulateAccuracy, round(loss.item(),4) ) )
    scheduler.step(trainingLoss)
    correctImages, totalImages, allLabels, allPredicted = testModel(testDatasetLoader, net)
    testAccuracy = round((correctImages/totalImages)*100,2)

    print('='*10)
    
    if testAccuracy > bestAccuracy:
      bestAccuracy = testAccuracy
      bestPredicted = allPredicted
      bestNet = net

  return bestAccuracy, bestPredicted, allLabels, bestNet
'''

'\n### Train and test helper function ###\ndef testModel(testDatasetLoader, net):\n  net.eval()\n  correctImages = 0\n  totalImages = 0\n  allLabels = []\n  allPredicted = []\n  testingProgressbar = tqdm(enumerate(testDatasetLoader), total=len(testDatasetLoader), ncols=100)\n  with torch.no_grad():\n    for batchIdx, batchData in testingProgressbar:\n      images, labels = batchData\n      \n      images, labels = images.cuda(), labels.cuda()\n      outputs = net(images)\n      _, predicted = torch.max(outputs, 1)\n\n      correctImages += (predicted == labels).sum().item()\n      totalImages += labels.size(0)\n\n      accumulateAccuracy = round((correctImages/totalImages)*100,4)\n      testingProgressbar.set_description("Testing accuracy: {}".format(accumulateAccuracy ) )\n    \n      allLabels.append(labels)\n      allPredicted.append(predicted)\n  allLabels = torch.cat(allLabels).cpu().numpy()\n  allPredicted = torch.cat(allPredicted).cpu().numpy()\n  return correctImages, totalImag

In [ ]:
'''
bestAccuracy, bestPredicted, allLabels, bestNet = trainAndTestModel(??)
'''

'\nbestAccuracy, bestPredicted, allLabels, bestNet = trainAndTestModel(??)\n'

## Classify on validation set and send result to server!

In [ ]:
'''
from PIL import Image
import json
import requests

class ImageFolderWithPaths(Dataset):
    def __init__(self, root_dir, transform=None):
        self.imageFileNames = sorted(glob.glob(root_dir+'/*.jpg'))
        self.transform = transform
    
    def __getitem__(self, idx):
        imageData = Image.open(self.imageFileNames[idx])
        imageFileName = os.path.basename(self.imageFileNames[idx])
        if self.transform is not None:
            imageData = self.transform(imageData)
        return imageFileName, imageData.unsqueeze(0)
    def __len__(self):
        return len(self.imageFileNames)



def generatePredictedResults(valDataset, net):
    net.eval()
    predictedResults = {}
    with torch.no_grad():
        for imageFileName, imageData in tqdm(valDataset, ncols=100):
            imageData = imageData.cuda()
            outputs = net(imageData)
            _, predicted = torch.max(outputs, 1)
            #print(imageFileName, predicted.item())
            predictedResults[imageFileName] =  foodTrainDataset.classes[predicted.item()]
    return predictedResults

def sendResult(predictedResults,studentID=5630008021):
    sendDict = { 'studentID':studentID, 'results':  predictedResults }
    response = requests.post('https://www.piclab.ai/classes/cv2021/lab8/scoreboard/submit',headers={'Content-Type': 'application/json' }, json=sendDict)
    return response.text
'''

"\nfrom PIL import Image\nimport json\nimport requests\n\nclass ImageFolderWithPaths(Dataset):\n    def __init__(self, root_dir, transform=None):\n        self.imageFileNames = sorted(glob.glob(root_dir+'/*.jpg'))\n        self.transform = transform\n    \n    def __getitem__(self, idx):\n        imageData = Image.open(self.imageFileNames[idx])\n        imageFileName = os.path.basename(self.imageFileNames[idx])\n        if self.transform is not None:\n            imageData = self.transform(imageData)\n        return imageFileName, imageData.unsqueeze(0)\n    def __len__(self):\n        return len(self.imageFileNames)\n\n\n\ndef generatePredictedResults(valDataset, net):\n    net.eval()\n    predictedResults = {}\n    with torch.no_grad():\n        for imageFileName, imageData in tqdm(valDataset, ncols=100):\n            imageData = imageData.cuda()\n            outputs = net(imageData)\n            _, predicted = torch.max(outputs, 1)\n            #print(imageFileName, predicted.item()

Example Result JSON <br>
resultDict = { 
  'studentID': 555555555521,
  'results':{
    '1.jpg':'ต้มเลือดหมู',
    '2.jpg':'บะหมี่แห้ง',
    ...
  }
}


In [ ]:
'''
foodTestDataset = ImageFolderWithPaths('test', transform=transformTest)
predictedResults = generatePredictedResults(foodTestDataset, bestNet)
print(sendResult(predictedResults, studentID=YOUR_STUDENT_ID_WITH21))
'''

"\nfoodTestDataset = ImageFolderWithPaths('test', transform=transformTest)\npredictedResults = generatePredictedResults(foodTestDataset, bestNet)\nprint(sendResult(predictedResults, studentID=YOUR_STUDENT_ID_WITH21))\n"

## Refactor code

In [ ]:
torch.manual_seed(2)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(2)
random_state = 42

In [ ]:
class config:

    classes = ['honey toast', 'กระเพาะปลา', 'ก๋วยจั๊บ', 'ก๋วยจั๊บญวน', 'ก๋วยเตี๋ยวคั่วไก่', 'ก๋วยเตี๋ยวเรือ', 'ขนมถ้วย',
               'ขนมปัง', 'ขนมปังปิ้ง', 'ขนมปังสังขยา', 'ขาหมูเยอรมัน', 'ข้าวผัดกระเทียม', 'ข้าวหน้าเนื้อ', 'ข้าวหน้าเป็ด',
               'ข้าวหมกไก่', 'ข้าวหมูกรอบ', 'ชาบู', 'ซุปเห็ด', 'ต้มเลือดหมู', 'ทาโกะยากิ', 'น้ำตกหมู', 'น้ำพริกไข่ปู',
               'บะหมี่แห้ง', 'ปลากระพงนึ่งมะนาว', 'ปอเปี๊ยะทอด', 'ปูนิ่มทอดกระเทียม', 'ปูผัดผงกะหรี่', 'ปูม้านึ่ง', 'ยำถั่วพลู',
               'ยำปลาดุกฟู', 'ยำสาหร่าย', 'ยำหมูยอ', 'ยำแซลมอน', 'สปาเก็ตตี้ขี้เมาทะเล', 'สปาเก็ตตี้คาโบนาร่า', 'สเต็กหมู',
               'หมูมะนาว', 'ออส่วน', 'ฮะเก๋า', 'เกี๊ยวทอด', 'เค้ก', 'เนื้อย่าง', 'เป็ดพะโล้', 'เป็ดย่าง', 'แกงคั่วหอยขม',
               'แกงส้มชะอมกุ้ง', 'ไข่กระทะ', 'ไข่ตุ๋น', 'ไส้อั่ว', 'ไอศกรีม']
    num_classes = len(classes)

    mean_imagenet = (0.485, 0.456, 0.406)
    std_imagenet  = (0.229, 0.224, 0.225)

    model_name = 'tf_efficientnetv2_s'

    if model_name == 'tf_efficientnetv2_s':
        image_size = 300
    elif model_name in ['tf_efficientnetv2_m', 'tf_efficientnetv2_l']:
        # image_size = 384
        image_size = 224
    elif model_name == 'tf_efficientnetv2_b3':
        image_size = 240
    elif model_name in ['convnext_large_in22k', 'convnext_xlarge_in22k']:
        image_size = 224
    elif model_name == 'vit_base_patch16_224':
        image_size = 224

    batch_size = 16

    lr = 0.00005
    gamma = 0.97

In [ ]:
class FoodDataset(Dataset):

    def __init__(self, image_paths, labels, image_size=224, dataset='train', transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.image_size = image_size
        self.dataset = dataset
        
        assert dataset in ['train', 'validate', 'test']
        if transform == None:
            self.transform = self.get_transform()
        else:
            self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = self.load_image(image_path=image_path)
        image = self.transform(image=image)['image']

        return image, label
    
    def load_image(self, image_path):
        image = np.array(Image.open(image_path).convert('RGB'))
        return image

    def get_transform(self):
        if self.dataset == 'train':
            transform = albumentations.Compose([
                albumentations.OneOf([
                    albumentations.HorizontalFlip(p=1),
                    albumentations.VerticalFlip(p=1),
                    albumentations.Rotate(limit=360, p=1),                  
                ], p=0.5),
                albumentations.RandomCrop(height=self.image_size, width=self.image_size, p=0.5),
                albumentations.ShiftScaleRotate(border_mode=cv2.BORDER_CONSTANT, scale_limit=0.3,
                                                rotate_limit=(10, 30), p=0.5),
                albumentations.MotionBlur(p=0.5),
                albumentations.OpticalDistortion(p=0.5),
                albumentations.GaussNoise(p=0.5),
                albumentations.Resize(width=self.image_size, height=self.image_size,
                                      interpolation=cv2.INTER_CUBIC),
                albumentations.Normalize(mean=config.mean_imagenet, std=config.std_imagenet),
                albumentations.pytorch.ToTensor(),             
            ])
        else:
            transform = albumentations.Compose([
                albumentations.Resize(width=self.image_size, height=self.image_size,
                                      interpolation=cv2.INTER_CUBIC),
                albumentations.Normalize(mean=config.mean_imagenet, std=config.std_imagenet),
                albumentations.pytorch.ToTensor(),                               
            ])
        
        return transform


def get_train_data():
    train_paths  = []
    train_labels = []
    dataset = 'train'

    for cls in os.listdir(dataset):
        if cls == '.DS_Store':
            continue
        image_paths = os.listdir(os.path.join(dataset, cls))
        image_paths = [os.path.join(dataset, cls, image_path) for image_path in image_paths]
        train_paths  += image_paths
        train_labels += [config.classes.index(cls)] * len(image_paths)

    return train_paths, train_labels


def get_dataloader():
    train_paths, train_labels = get_train_data()
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        train_paths, train_labels, test_size=0.2, stratify=train_labels, random_state=random_state
    )

    train_dataset = FoodDataset(image_paths=train_paths, labels=train_labels, image_size=config.image_size,
                                dataset='train', transform=None)
    val_dataset   = FoodDataset(image_paths=val_paths, labels=val_labels, image_size=config.image_size,
                                dataset='validate', transform=None)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=config.batch_size, shuffle=True, 
                                  num_workers=4, pin_memory=True)
    val_dataloader   = DataLoader(dataset=val_dataset, batch_size=config.batch_size, shuffle=False,
                                  num_workers=4, pin_memory=True)

    return train_dataloader, val_dataloader

In [ ]:
!pip install timm

     |████████████████████████████████| 431 kB 13.9 MB/s 


In [ ]:
import timm


class Model(nn.Module):

    def __init__(self, model_name):
        super(Model, self).__init__()

        self.cnn = timm.create_model(model_name, pretrained=True)

        if model_name in ['tf_efficientnetv2_s', 'tf_efficientnetv2_m', 'tf_efficientnetv2_l',
                          'tf_efficientnetv2_xl_in21k']:
            in_features = 1280
        elif model_name in ['tf_efficientnetv2_b3']:
            in_features = 1536
        elif 'convnext' in model_name:
            in_features = 2048
        elif model_name in ['vit_base_patch16_224']:
            in_features = 768
        
        if 'convnext' in model_name:
            self.cnn.head.fc = nn.Linear(in_features=in_features, out_features=config.num_classes)
        elif 'tf_efficientnetv2' in model_name:
            self.cnn.classifier = nn.Linear(in_features=in_features, out_features=config.num_classes)
        elif 'vit' in model_name:
            self.head = nn.Linear(in_features=in_features, out_features=config.num_classes)

    def forward(self, x):
        x = self.cnn(x)
        return x
    
    def freeze(self):
        for name, parameter in self.named_parameters():
            if not 'classifier' in name:
                parameter.requires_grad = False

    def unfreeze(self):
        for parameter in self.parameters():
            parameter.requires_grad = True

In [ ]:
def accuracy(y_pred, y_true):
    y_pred = nn.functional.softmax(y_pred, dim=1)
    _, top_class = y_pred.topk(1, dim=1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.sum(equals.type(torch.FloatTensor))


class Trainer:

    def __init__(self, criterion, optimizer, scheduler=None):
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f'Device: {self.device}')

    def train_batch_loop(self, model, dataloader):
        train_loss = 0.0
        train_acc  = 0.0

        for inputs, labels in tqdm(dataloader):
            inputs, labels = inputs.to(self.device), labels.to(self.device)

            logits = model(inputs)

            loss = self.criterion(logits, labels)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            train_loss += loss.item()
            train_acc += accuracy(logits, labels)
        
        return train_loss / len(dataloader), train_acc / len(dataloader.dataset)

    def valid_batch_loop(self, model, dataloader):
        val_loss = 0.0
        val_acc  = 0.0

        with torch.no_grad():
            for inputs, labels in tqdm(dataloader):
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                logits = model(inputs)

                loss = self.criterion(logits, labels)

                val_loss += loss.item()
                val_acc += accuracy(logits, labels)
          
        return val_loss / len(dataloader), val_acc / len(dataloader.dataset)

    def train(self, model, train_dataloader, val_dataloader, epochs, save_weights_path='model.pt'):
        model = nn.DataParallel(model)
        model.to(self.device)

        val_min_loss = np.Inf

        for epoch in range(epochs):
            model.train()
            train_loss, train_acc = self.train_batch_loop(model=model, dataloader=train_dataloader)

            model.eval()
            val_loss, val_acc = self.valid_batch_loop(model=model, dataloader=val_dataloader)

            if self.scheduler is not None:
                self.scheduler.step(val_loss)

            if val_loss <= val_min_loss:
                print('val_loss improved, saving model weight to {}'.format(save_weights_path))
                torch.save(model.module.state_dict(), save_weights_path)
                val_min_loss = val_loss

            print('Epoch : {}'.format(epoch+1))
            print('   train_loss : {:.6f} train_acc : {:.6f}'.format(train_loss, train_acc))
            print('   val_loss   : {:.6f} val_acc   : {:.6f}'.format(val_loss, val_acc))

In [ ]:
train_dataloader, val_dataloader = get_dataloader()

model = Model(model_name=config.model_name)

optimizer = torch.optim.Adam(params=model.parameters(), lr=config.lr)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=config.gamma)

trainer = Trainer(criterion=criterion, optimizer=optimizer, scheduler=scheduler)
trainer.train(model=model, train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
              epochs=20, save_weights_path=f'{config.model_name}.pt')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-effv2-weights/tf_efficientnetv2_s-eb54923e.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnetv2_s-eb54923e.pth


Device: cuda


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

val_loss improved, saving model weight to tf_efficientnetv2_s.pt
Epoch : 1
   train_loss : 2.352517 train_acc : 0.478250
   val_loss   : 0.810365 val_acc   : 0.824500


...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

Epoch : 12
   train_loss : 0.186648 train_acc : 0.947563
   val_loss   : 0.332968 val_acc   : 0.921000


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
import json
import requests


class FoodTestset(Dataset):
    
    def __init__(self, image_size=224):
        self.image_paths = sorted(glob.glob('test'+'/*.jpg'))
        self.image_size = image_size
        self.transform = self.get_transform()
    
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image_name = os.path.basename(image_path)
        image = self.load_image(image_path=image_path)
        image = self.transform(image=image)['image']

        return image, image_name

    def load_image(self, image_path):
        image = np.array(Image.open(image_path).convert('RGB'))
        return image
    
    def get_transform(self):
        transform = albumentations.Compose([
            albumentations.Resize(width=self.image_size, height=self.image_size,
                                  interpolation=cv2.INTER_CUBIC),
            albumentations.Normalize(mean=config.mean_imagenet, std=config.std_imagenet),
            albumentations.pytorch.ToTensor(),                               
        ])
        
        return transform


def generate_predicted_results(test_dataset, model):
    model.eval()
    predicted_result = dict()

    with torch.no_grad():
        for image, image_name in tqdm(test_dataset):
            image = image.cuda()
            image = torch.unsqueeze(image, 0)
            outputs = model(image)

            _, predicted = torch.max(outputs, 1)
            predicted_result[image_name] = config.classes[predicted.item()]

    return predicted_result

def send_result(predicted_result, student_id=6130112321):
    send_dict = dict(studentID=student_id, results=predicted_result)
    response = requests.post('https://www.piclab.ai/classes/cv2021/lab8/scoreboard/submit',
                             headers={'Content-Type': 'application/json'}, json=send_dict)
    return response.text

In [ ]:
model.load_state_dict(torch.load(f'{config.model_name}.pt'))
test_dataset = FoodTestset(image_size=config.image_size)

In [ ]:
predicted_result = generate_predicted_results(test_dataset=test_dataset, model=model)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
send_result(predicted_result=predicted_result, student_id=6130112321)

'{"accuracy":"91.36","status":"SUCCESS"}\n'